In [112]:
import yfinance as yf
import pandas as pd 
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models, expected_returns
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [113]:
tickers = [
    'D05.SI', 'O39.SI', 'U11.SI', 'Z74.SI', 'F34.SI', 'S07.SI', 
    'C6L.SI', 'Q0F.SI', 'C38U.SI', 'BN4.SI', 'S63.SI', 
    'Y92.SI', 'A17U.SI', 'G13.SI', 'C07.SI'
]
data = yf.download(tickers, start="2015-01-01", end="2023-12-31")
adj_close_prices = data['Adj Close']

[*********************100%%**********************]  15 of 15 completed


In [114]:
adj_close_prices.head(10)

Ticker,A17U.SI,BN4.SI,C07.SI,C38U.SI,C6L.SI,D05.SI,F34.SI,G13.SI,O39.SI,Q0F.SI,S07.SI,S63.SI,U11.SI,Y92.SI,Z74.SI
Date,,,,,,,,,,,,,,,
2015-01-02,1.397633,3.277845,30.608868,1.267565,6.473314,12.136217,2.500496,0.841017,7.215527,1.356088,9.583110,2.358384,17.901175,0.000127,2.709046
2015-01-05,1.403456,3.214952,29.876957,1.273778,6.495733,11.863894,2.469910,0.829227,7.112448,1.345003,9.583110,2.344552,17.499475,0.000124,2.702082
2015-01-06,1.380163,3.078067,29.203588,1.255138,6.546173,11.739572,2.439322,0.809577,6.981883,1.322833,9.583110,2.296139,17.032042,0.000124,2.674226
2015-01-07,1.397633,3.089166,29.079165,1.292419,6.703103,11.721810,2.424029,0.801717,6.995624,1.322833,9.583110,2.316887,17.119686,0.000129,2.688154
2015-01-08,1.397633,3.133561,29.203588,1.261351,6.736730,11.887571,2.477556,0.809577,7.215527,1.341308,9.583110,2.323804,17.543299,0.000129,2.743868
2015-01-09,1.415104,3.096565,29.291418,1.279992,6.815194,11.917171,2.485203,0.805647,7.160552,1.330223,9.583110,2.337636,17.075865,0.000131,2.709046
2015-01-12,1.438397,3.066968,29.108437,1.292419,7.005751,11.857969,2.477556,0.801717,7.167422,1.345003,9.583110,2.337636,17.148901,0.000131,2.716011
2015-01-13,1.473338,3.022573,28.969370,1.311060,6.988936,11.869811,2.454615,0.797787,7.174294,1.326527,9.583110,2.330720,17.061256,0.000132,2.709046
2015-01-14,1.409280,2.996676,28.442394,1.304846,6.983333,11.828371,2.462262,0.793857,7.064344,1.326527,9.583110,2.323804,16.966312,0.000134,2.729939


In [115]:
# Splitting the data into training and testing sets
train_data = adj_close_prices.loc['2015-01-01':'2022-12-31']
test_data = adj_close_prices.loc['2023-01-01':'2023-12-31']

In [116]:
print(train_data.shape)
print(test_data.shape)

(2010, 15)
(249, 15)


In [117]:
mu = mean_historical_return(train_data)
S = CovarianceShrinkage(train_data).ledoit_wolf()

In [118]:
print(mu)
print(S)

Ticker
A17U.SI    0.076911
BN4.SI     0.037068
C07.SI    -0.012543
C38U.SI    0.050844
C6L.SI    -0.026131
D05.SI     0.113175
F34.SI     0.060393
G13.SI     0.011654
O39.SI     0.059447
Q0F.SI     0.023040
S07.SI    -0.047528
S63.SI     0.039466
U11.SI     0.062642
Y92.SI     1.480270
Z74.SI    -0.010518
dtype: float64
Ticker    A17U.SI    BN4.SI    C07.SI   C38U.SI    C6L.SI    D05.SI    F34.SI  \
Ticker                                                                          
A17U.SI  0.055077  0.011417  0.012771  0.021548  0.011821  0.010994  0.012121   
BN4.SI   0.011417  0.069403  0.020378  0.011645  0.013607  0.018971  0.017795   
C07.SI   0.012771  0.020378  0.083340  0.014682  0.015313  0.017436  0.016234   
C38U.SI  0.021548  0.011645  0.014682  0.061073  0.014486  0.011993  0.012281   
C6L.SI   0.011821  0.013607  0.015313  0.014486  0.064177  0.015995  0.011925   
D05.SI   0.010994  0.018971  0.017436  0.011993  0.015995  0.054286  0.014121   
F34.SI   0.012121  0.017795  0

In [119]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

In [120]:
print(weights)

OrderedDict([('A17U.SI', 0.1583759727083442), ('BN4.SI', 0.0), ('C07.SI', 0.0), ('C38U.SI', 0.0), ('C6L.SI', 0.0), ('D05.SI', 0.3620737936604053), ('F34.SI', 0.0388472974897319), ('G13.SI', 0.0), ('O39.SI', 0.0), ('Q0F.SI', 0.0), ('S07.SI', 0.0), ('S63.SI', 0.0), ('U11.SI', 0.0), ('Y92.SI', 0.4407029361415187), ('Z74.SI', 0.0)])


In [121]:
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

OrderedDict([('A17U.SI', 0.15838), ('BN4.SI', 0.0), ('C07.SI', 0.0), ('C38U.SI', 0.0), ('C6L.SI', 0.0), ('D05.SI', 0.36207), ('F34.SI', 0.03885), ('G13.SI', 0.0), ('O39.SI', 0.0), ('Q0F.SI', 0.0), ('S07.SI', 0.0), ('S63.SI', 0.0), ('U11.SI', 0.0), ('Y92.SI', 0.4407), ('Z74.SI', 0.0)])


In [122]:
performance = ef.portfolio_performance(verbose=True)

Expected annual return: 70.8%
Annual volatility: 46.8%
Sharpe Ratio: 1.47


### 6) You will backtest the performance through these two approaches:
a)
Using the weights, we get the daily portfolio return series everyday by simply multiplying the weights on the regular returns for each stock for the period of 2023. We compute the total portfolio return and standard deviation from this return series. (and also the sharpe ratio).

In [123]:
# Calculate returns and append new columns for each stock
for ticker in test_data.columns:
    test_data[f'{ticker}_returns'] = test_data[ticker].pct_change()

/var/folders/l_/xlcq7dfs5hb1cq7xjkkmbzr40000gn/T/ipykernel_51069/3548479454.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[f'{ticker}_returns'] = test_data[ticker].pct_change()
/var/folders/l_/xlcq7dfs5hb1cq7xjkkmbzr40000gn/T/ipykernel_51069/3548479454.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[f'{ticker}_returns'] = test_data[ticker].pct_change()
/var/folders/l_/xlcq7dfs5hb1cq7xjkkmbzr40000gn/T/ipykernel_51069/3548479454.py:3: SettingWithCopyWarning: 
A value is trying to 

In [130]:
def backtest1(weights):
    return_columns = [f"{k}_returns" for k in weights.keys()]
    returns_data = test_data[return_columns]
    weights_series = pd.Series(weights)
    portfolio_daily_returns = []
    
    for index, row in returns_data.iterrows():
        daily_return = 0
        for col in return_columns:
            stock_symbol = col.replace('_returns', '') 
            daily_return += row[col] * weights_series[stock_symbol]
        portfolio_daily_returns.append(daily_return)
    
    print(len(portfolio_daily_returns))
    print(len(returns_data.index))
    
    portfolio_daily_returns = pd.Series(portfolio_daily_returns, index=returns_data.index)
    cumulative_return = (1 + test_data['portfolio_daily_returns']).prod() - 1
    standard_deviation = test_data['portfolio_daily_returns'].std() * (252**0.5)
    
    return cumulative_return, standard_deviation

In [131]:
returns1, sd1 = backtest1(cleaned_weights)
print(f"Total Cumulative Portfolio Return: {returns1 * 100:.2f}%")
print(f"Annualized Standard Deviation: {sd1:.2f}")

249
249
Total Cumulative Portfolio Return: 90.91%
Annualized Standard Deviation: 0.80


In [126]:
# Identify all return columns based on the format "<stock_symbol>_returns"
return_columns = [f"{k}_returns" for k in weights.keys()]

# Extract only the return columns from your test_data DataFrame
returns_data = test_data[return_columns]

# Convert weights to a Pandas Series for consistent handling
weights_series = pd.Series(weights)

# Initialize an empty list to store portfolio daily returns
portfolio_daily_returns = []

# Loop through each row in the DataFrame
for index, row in returns_data.iterrows():
    daily_return = 0
    for col in return_columns:
        stock_symbol = col.replace('_returns', '')  # Extract the stock symbol from the column name
        # Calculate weighted return for the day and sum it
        daily_return += row[col] * weights_series[stock_symbol]
    # Append the result to the list
    portfolio_daily_returns.append(daily_return)

# Convert list to Pandas Series
portfolio_daily_returns = pd.Series(portfolio_daily_returns, index=returns_data.index)

test_data['portfolio_daily_returns'] = portfolio_daily_returns

/var/folders/l_/xlcq7dfs5hb1cq7xjkkmbzr40000gn/T/ipykernel_51069/634748818.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['portfolio_daily_returns'] = portfolio_daily_returns


In [127]:
# Calculate the cumulative return using the product of (1 + daily returns)
cumulative_return = (1 + test_data['portfolio_daily_returns']).prod() - 1

# Calculate the annualized standard deviation of the daily returns
# Multiply by the square root of the number of trading days (typically 252) to annualize
standard_deviation = test_data['portfolio_daily_returns'].std() * (252**0.5)

# Display the computed total cumulative return and annualized standard deviation
print(f"Total Cumulative Portfolio Return: {cumulative_return * 100:.2f}%")
print(f"Annualized Standard Deviation: {standard_deviation:.2f}")

Total Cumulative Portfolio Return: 90.91%
Annualized Standard Deviation: 0.80


In [97]:
# Assuming 'test_data' is your DataFrame with initial prices and possibly other data
# Ensure only the relevant columns, those that are stock symbols in your weights, are selected
relevant_columns = [col for col in test_data.columns if col in cleaned_weights]

# Extract the initial prices for each relevant stock from the first row of the DataFrame
initial_prices = test_data.loc[test_data.index[0], relevant_columns]

# Print the initial prices to verify correct extraction
print("Initial Prices:")
print(initial_prices)

# Adjust the order of weights to exactly match the order of 'initial_prices'
adjusted_weights = {ticker: cleaned_weights[ticker] for ticker in relevant_columns}

# Print adjusted weights to verify alignment
print("Adjusted Weights:")
print(adjusted_weights)

# Set up the Discrete Allocation
total_portfolio_value = 1000000  # $1,000,000
da = DiscreteAllocation(adjusted_weights, initial_prices, total_portfolio_value=total_portfolio_value)

# Get the discrete allocation of each stock
allocation, leftover = da.lp_portfolio()
print(f"Discrete allocation: {allocation}")
print(f"Funds remaining: ${leftover:.2f}")

Initial Prices:
Ticker
A17U.SI     2.523124
BN4.SI      4.387392
C07.SI     27.320511
C38U.SI     1.873024
C6L.SI      5.203522
D05.SI     28.496010
F34.SI      3.933333
G13.SI      0.917571
O39.SI     11.490488
Q0F.SI      1.591709
S07.SI      6.630000
S63.SI      3.230103
U11.SI     29.056496
Y92.SI      0.175774
Z74.SI      2.470665
Name: 2023-01-03 00:00:00, dtype: float64
Adjusted Weights:
{'A17U.SI': 0.15838, 'BN4.SI': 0.0, 'C07.SI': 0.0, 'C38U.SI': 0.0, 'C6L.SI': 0.0, 'D05.SI': 0.36207, 'F34.SI': 0.03885, 'G13.SI': 0.0, 'O39.SI': 0.0, 'Q0F.SI': 0.0, 'S07.SI': 0.0, 'S63.SI': 0.0, 'U11.SI': 0.0, 'Y92.SI': 0.4407, 'Z74.SI': 0.0}
Discrete allocation: {'A17U.SI': 62771, 'D05.SI': 12706, 'F34.SI': 9877, 'Y92.SI': 2507197}
Funds remaining: $1.05


In [98]:
needed_columns = list(allocation.keys())
for stock, shares in allocation.items():
    if stock in test_data.columns:  # Check if the stock's price data is available
        test_data[f'{stock}_value'] = test_data[stock] * shares
    else:
        print(f"Warning: Price data for {stock} not found in test_data")

print(test_data.head())

Ticker       A17U.SI    BN4.SI     C07.SI   C38U.SI    C6L.SI     D05.SI  \
Date                                                                       
2023-01-03  2.523124  4.387392  27.320511  1.873024  5.203522  28.496010   
2023-01-04  2.532332  4.357217  27.175344  1.854571  5.194044  28.613762   
2023-01-05  2.569166  4.417566  26.710810  1.882251  5.250913  29.252985   
2023-01-06  2.513915  4.381357  26.855976  1.845344  5.241436  28.992249   
2023-01-09  2.550749  4.411531  27.039856  1.863797  5.279348  29.648293   

Ticker        F34.SI    G13.SI     O39.SI    Q0F.SI  ...  S07.SI_returns  \
Date                                                 ...                   
2023-01-03  3.933333  0.917571  11.490488  1.591709  ...             NaN   
2023-01-04  3.914193  0.907912  11.537427  1.583105  ...        0.010558   
2023-01-05  3.952474  0.927230  11.678241  1.583105  ...        0.019403   
2023-01-06  3.962044  0.936888  11.734568  1.617520  ...       -0.024890   
2023-01-09 

/var/folders/l_/xlcq7dfs5hb1cq7xjkkmbzr40000gn/T/ipykernel_51069/2974385826.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[f'{stock}_value'] = test_data[stock] * shares
/var/folders/l_/xlcq7dfs5hb1cq7xjkkmbzr40000gn/T/ipykernel_51069/2974385826.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[f'{stock}_value'] = test_data[stock] * shares
/var/folders/l_/xlcq7dfs5hb1cq7xjkkmbzr40000gn/T/ipykernel_51069/2974385826.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [99]:
# Create a list of the stock value columns
value_columns = []
value_columns = [col for col in test_data.columns if '_value' in col]
print(value_columns)

# Calculate the daily portfolio value by summing all the stock value columns across each row
test_data['daily_portfolio_value'] = test_data[value_columns].sum(axis=1)

# Calculate daily returns as the percentage change in 'daily_portfolio_value'
test_data['portfolio_daily_returns_2'] = test_data['daily_portfolio_value'].pct_change()

['A17U.SI_value', 'D05.SI_value', 'F34.SI_value', 'Y92.SI_value']


/var/folders/l_/xlcq7dfs5hb1cq7xjkkmbzr40000gn/T/ipykernel_51069/3903265611.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['daily_portfolio_value'] = test_data[value_columns].sum(axis=1)
/var/folders/l_/xlcq7dfs5hb1cq7xjkkmbzr40000gn/T/ipykernel_51069/3903265611.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['portfolio_daily_returns_2'] = test_data['daily_portfolio_value'].pct_change()


In [102]:
# Calculate the cumulative return using the product of (1 + daily returns)
cumulative_return_2 = (1 + test_data['portfolio_daily_returns_2']).prod() - 1

# Calculate the annualized standard deviation of the daily returns
# Multiply by the square root of the number of trading days (typically 252) to annualize
standard_deviation_2 = test_data['portfolio_daily_returns_2'].std() * (252**0.5)

# Display the computed total cumulative return and annualized standard deviation
print(f"Total Cumulative Portfolio Return: {cumulative_return_2 * 100:.2f}%")
print(f"Annualized Standard Deviation: {standard_deviation_2:.2f}")

Total Cumulative Portfolio Return: 87.33%
Annualized Standard Deviation: 0.82


### 7) Repeat the experiment with sector constraints.

#### Sector Constraints 1

In [104]:
# Sorted stock symbols
sorted_tickers = sorted([
    'A17U.SI', 'BN4.SI', 'C07.SI', 'C38U.SI', 'C6L.SI', 'D05.SI', 
    'F34.SI', 'G13.SI', 'O39.SI', 'Q0F.SI', 'S07.SI', 'S63.SI', 
    'U11.SI', 'Y92.SI', 'Z74.SI'
])

# Sector mapper with real sector values
sector_mapper = {
    'A17U.SI': 'Real Estate',   # Real Estate
    'BN4.SI': 'Industrials',    # Industrials
    'C07.SI': 'Consumer Goods', # Consumer Goods
    'C38U.SI': 'Utilities',     # Utilities
    'C6L.SI': 'Industrials',    # Industrials
    'D05.SI': 'Financials',     # Financials
    'F34.SI': 'Industrials',    # Industrials
    'G13.SI': 'Technology',     # Technology
    'O39.SI': 'Financials',     # Financials
    'Q0F.SI': 'Utilities',      # Utilities
    'S07.SI': 'Industrials',    # Industrials
    'S63.SI': 'Industrials',    # Industrials
    'U11.SI': 'Real Estate',    # Real Estate
    'Y92.SI': 'Technology',     # Technology
    'Z74.SI': 'Technology'      # Technology
}

# Sector weight boundaries
sector_lower = {
    'Financials': 0.10,
    'Industrials': 0.10,
    'Technology': 0.10,
    'Utilities': 0.10,
    'Consumer Goods': 0.05,
    'Real Estate': 0.05
}

sector_upper = {
    'Financials': 0.40,
    'Industrials': 0.30,
    'Technology': 0.30,
    'Utilities': 0.20,
    'Consumer Goods': 0.20,
    'Real Estate': 0.20
}

In [109]:
# Setup Efficient Frontier
ef2 = EfficientFrontier(mu, S)
ef2.add_sector_constraints(sector_mapper, sector_lower, sector_upper)

ef2.max_sharpe()
print(ef2.clean_weights())

OrderedDict([('A17U.SI', 0.15101), ('BN4.SI', 0.0), ('C07.SI', 0.05), ('C38U.SI', 0.03701), ('C6L.SI', 0.0), ('D05.SI', 0.29899), ('F34.SI', 0.1), ('G13.SI', 0.0), ('O39.SI', 0.0), ('Q0F.SI', 0.06299), ('S07.SI', 0.0), ('S63.SI', 0.0), ('U11.SI', 0.0), ('Y92.SI', 0.3), ('Z74.SI', 0.0)])


In [ ]:
# Identify all return columns based on the format "<stock_symbol>_returns"
return_columns = [f"{k}_returns" for k in weights.keys()]

# Extract only the return columns from your test_data DataFrame
returns_data = test_data[return_columns]

# Convert weights to a Pandas Series for consistent handling
weights_series = pd.Series(weights)

# Initialize an empty list to store portfolio daily returns
portfolio_daily_returns = []

# Loop through each row in the DataFrame
for index, row in returns_data.iterrows():
    daily_return = 0
    for col in return_columns:
        stock_symbol = col.replace('_returns', '')  # Extract the stock symbol from the column name
        # Calculate weighted return for the day and sum it
        daily_return += row[col] * weights_series[stock_symbol]
    # Append the result to the list
    portfolio_daily_returns.append(daily_return)

# Convert list to Pandas Series
portfolio_daily_returns = pd.Series(portfolio_daily_returns, index=returns_data.index)

#### Sector Constraints 2

#### Sector Constraints 3

In [100]:
test_data.to_csv("test_data.csv")